In [9]:
import numpy as np
import torch

data = torch.from_numpy(np.load('./res/tmp/plenoptic/coffee_martini/poses_bounds.npy'))
data.shape

torch.Size([18, 17])

In [10]:
def get_poses_raw(data):
    mat, close, far = data[:, :-2].reshape((-1, 3, 5)), data[:, -2], data[:, -1]
    T, mat2 = mat[:, :, :-1], mat[:, :, -1:]
    h, w, f = mat2[:, 0, 0], mat2[:, 1, 0], mat2[:, 2, 0]

    return T, (h, w), (close, far), f

T, (h, w), (c, f), F = get_poses_raw(data)

T.shape, (h.shape, w.shape), (c.shape, f.shape), F.shape


(torch.Size([18, 3, 4]),
 (torch.Size([18]), torch.Size([18])),
 (torch.Size([18]), torch.Size([18])),
 torch.Size([18]))

In [11]:
data[0, :], T[0, :, :], (h[0], w[0]), (c[0], f[0]), F[0]


(tensor([-2.1732e-02,  9.9435e-01, -1.0391e-01,  4.4396e-01,  2.0280e+03,
          9.9975e-01,  2.1133e-02, -6.8671e-03, -1.1035e+00,  2.7040e+03,
         -4.6324e-03, -1.0403e-01, -9.9456e-01, -3.4993e-01,  1.4608e+03,
          8.8314e+00,  1.0978e+02], dtype=torch.float64),
 tensor([[-0.0217,  0.9943, -0.1039,  0.4440],
         [ 0.9998,  0.0211, -0.0069, -1.1035],
         [-0.0046, -0.1040, -0.9946, -0.3499]], dtype=torch.float64),
 (tensor(2028., dtype=torch.float64), tensor(2704., dtype=torch.float64)),
 (tensor(8.8314, dtype=torch.float64), tensor(109.7754, dtype=torch.float64)),
 tensor(1460.7543, dtype=torch.float64))

In [ ]:
def colmap_poses_to_intrinsics_extrinsics(data):
    mat, close, far = data[:, :-2].reshape((-1, 3, 5)), data[:, -2], data[:, -1]
    T, mat2 = mat[:, :, :-1], mat[:, :, -1:]
    h, w, f = mat2[:, 0, 0], mat2[:, 1, 0], mat2[:, 2, 0]

    # Since we only have width, height and focal point, the intrinsics matrix will give imprecise results
    K = torch.zeros((T.shape[0], 3, 3))
    K[:, 0, 0], K[:, 1, 1], K[:, 2, 2], K[:, 0, 2], K[:, 1, 2] = f, f, 1, w / 2, h / 2

    return K, T, (h, w)

K, T, (h, w) = colmap_poses_to_intrinsics_extrinsics(data)

K.shape, T.shape, (h.shape, w.shape)


(torch.Size([18, 3, 3]),
 torch.Size([18, 3, 4]),
 (torch.Size([18]), torch.Size([18])))

In [26]:
K[0, :, :], T[0, :, :], (h[0], w[0])


(tensor([[1.4608e+03, 0.0000e+00, 1.3520e+03],
         [0.0000e+00, 1.4608e+03, 1.0140e+03],
         [0.0000e+00, 0.0000e+00, 1.0000e+00]]),
 tensor([[-0.0217,  0.9943, -0.1039,  0.4440],
         [ 0.9998,  0.0211, -0.0069, -1.1035],
         [-0.0046, -0.1040, -0.9946, -0.3499]], dtype=torch.float64),
 (tensor(2028., dtype=torch.float64), tensor(2704., dtype=torch.float64)))

In [ ]:
def load_data(basedir, factor=None, width=None, height=None):
    # Taken from repos/LLFF/llff/poses/pose_utils.py
    
    poses_arr = np.load(os.path.join(basedir, 'poses_bounds.npy'))
    poses = poses_arr[:, :-2].reshape([-1, 3, 5]).transpose([1,2,0])
    bds = poses_arr[:, -2:].transpose([1,0])
    
    img0 = [os.path.join(basedir, 'images', f) for f in sorted(os.listdir(os.path.join(basedir, 'images'))) \
            if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')][0]
    sh = imageio.imread(img0).shape
    
    sfx = ''
    
    # just resizes the images and sends to a new folder with suffix images_{}x{} or images_{}
    if factor is not None:
        sfx = '_{}'.format(factor)
        minify(basedir, factors=[factor])
        factor = factor
    elif height is not None:
        factor = sh[0] / float(height)
        width = int(sh[1] / factor)
        minify(basedir, resolutions=[[height, width]])
        sfx = '_{}x{}'.format(width, height)
    elif width is not None:
        factor = sh[1] / float(width)
        height = int(sh[0] / factor)
        minify(basedir, resolutions=[[height, width]])
        sfx = '_{}x{}'.format(width, height)
    else:
        factor = 1
    
    imgdir = os.path.join(basedir, 'images' + sfx)
    if not os.path.exists(imgdir):
        print( imgdir, 'does not exist, returning' )
        return
    
    imgfiles = [os.path.join(imgdir, f) for f in sorted(os.listdir(imgdir)) if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')]
    if poses.shape[-1] != len(imgfiles):
        print( 'Mismatch between imgs {} and poses {} !!!!'.format(len(imgfiles), poses.shape[-1]) )
        return
    
    # new data after resizing is width/factor, height/factor, focal/factor (factor 2 halves the image, 4 gets 1/4th of the size, and so on)
     sh = imageio.imread(imgfiles[0]).shape
    poses[:2, 4, :] = np.array(sh[:2]).reshape([2, 1])
    poses[2, 4, :] = poses[2, 4, :] * 1./factor
    
    return poses, bds
